In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, recall_score, precision_score, f1_score
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def neural_network_tabular(df):
    # Dividir em features (X) e alvo (y)
    X = df.drop('diagnostico', axis=1)  # Supondo que a coluna 'diagnostico' seja a coluna alvo
    y = df['diagnostico']
    
    # Configurar k-fold cross-validation
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    # Inicializar variáveis para armazenar métricas
    fold_accuracies = []
    fold_aucs = []
    fold_sensitivities = []
    fold_precisions = []
    fold_f1_scores = []
    
    mean_fpr = np.linspace(0, 1, 100)
    tpr_list = []

    for train_index, val_index in kfold.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Upsampling da classe minoritária
        X_train = pd.concat([X_train, y_train], axis=1)
        not_diagnostico = X_train[X_train.diagnostico == 0]
        diagnostico = X_train[X_train.diagnostico == 1]
        
        diagnostico_upsampled = resample(diagnostico, replace=True, n_samples=len(not_diagnostico), random_state=42)
        upsampled = pd.concat([not_diagnostico, diagnostico_upsampled])
        
        y_train = upsampled.diagnostico
        X_train = upsampled.drop('diagnostico', axis=1)

        # Padronização dos dados
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        
        # Definir o modelo
        model = Sequential([
            Dense(64, input_shape=(X_train.shape[1],), activation='relu'),  # Primeira camada oculta com 64 neurônios
            Dropout(0.5),  # Regularização
            Dense(32, activation='relu'),  # Segunda camada oculta com 32 neurônios
            Dropout(0.5),  # Regularização
            Dense(16, activation='relu'),  # Terceira camada oculta com 16 neurônios
            Dropout(0.5),  # Regularização
            Dense(1, activation='sigmoid')  # Saída binária
        ])
        
        # Compilar o modelo
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Implementar Early Stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Treinar o modelo
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping])
        
        # Avaliar o modelo
        y_pred_prob = model.predict(X_val).ravel()  # Previsão das probabilidades
        y_pred = (y_pred_prob > 0.5).astype("int32")  # Converter as previsões para 0 ou 1
        val_accuracy = accuracy_score(y_val, y_pred)
        
        # Calcular AUROC e outras métricas
        auc_score = roc_auc_score(y_val, y_pred_prob)
        fpr, tpr, _ = roc_curve(y_val, y_pred_prob)
        tpr_list.append(np.interp(mean_fpr, fpr, tpr))  # Interpolação para ROC
        
        sensitivity = recall_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        
        # Armazenar métricas
        fold_accuracies.append(val_accuracy)
        fold_aucs.append(auc_score)
        fold_sensitivities.append(sensitivity)
        fold_precisions.append(precision)
        fold_f1_scores.append(f1)

    # Plotar AUROC final
    mean_tpr = np.mean(tpr_list, axis=0)
    mean_tpr = np.append(mean_tpr, 1)  # Garantir que o valor final seja 1
    mean_auc = np.mean(fold_aucs)
    
    plt.figure(figsize=(8, 6))
    plt.plot(mean_fpr, mean_tpr, color='blue', label=f'Média AUROC = {mean_auc:.2f}')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()
    
    print(f"Mean Accuracy: {np.mean(fold_accuracies):.2f}")
    print(f"Mean Sensitivity: {np.mean(fold_sensitivities):.2f}")
    print(f"Mean Precision: {np.mean(fold_precisions):.2f}")
    print(f"Mean F1-Score: {np.mean(fold_f1_scores):.2f}")

    # Matriz de Confusão final
    cm = confusion_matrix(y_val, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()

# Exemplo de uso (df é o DataFrame com os dados)
# neural_network_tabular(df)


In [32]:
df = pd.read_csv('df_regressao.csv')  # Substitua pelo seu dataset
neural_network_tabular(df)

/home/grad/ccomp/23/matias.joao/miniconda3/envs/uti/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 

In [25]:
df = pd.read_csv('df_final.csv')  # Substitua pelo seu dataset
neural_network_tabular(df)

/home/grad/ccomp/23/matias.joao/miniconda3/envs/uti/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 5s 788us/step - accuracy: 0.9892 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 2/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 2s 718us/step - accuracy: 0.9906 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 3/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 2s 770us/step - accuracy: 0.9901 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 4/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 1s 648us/step - accuracy: 0.9891 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 5/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 1s 616us/step - accuracy: 0.9901 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 6/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 1s 668us/step - accuracy: 0.9905 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 7/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.9900 - loss: nan - val_accuracy: 0.9889 - val_loss: nan
Epoch 8/50
2041/2041 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - accuracy: 0.9896 - loss: nan - val_accuracy: 0.9889 - val_lo

ValueError: Input contains NaN.